<a href="https://colab.research.google.com/github/palaceIA/CICS_Project/blob/main/src/notebooks/albert/albert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning ALBERT v2
Esse jupyter faz parte na nossa pesquisa para o Congresso Internacional (CICS) , aqui você ira encontrar nosso experimento referente ao modelo ALBERT e os nossos devidos resultados. O dataset utlizado foi o "dair-ai/emotion" disponivel no HuggingFace .

## BASE ALBERT v2
Modelo pré-treinado em inglês usando um objetivo de modelagem de linguagem mascarada (MLM). Foi introduzido neste artigo e lançado pela primeira vez neste repositório . Este modelo, como todos os modelos ALBERT, não é caseado: não faz diferença entre inglês e inglês.

ALBERT é um modelo de transformadores pré-treinado em um grande corpus de dados em inglês de forma autossupervisionada. Isso significa que ele foi pré-treinado apenas nos textos brutos, sem humanos rotulando-os de forma alguma (é por isso que ele pode usar muitos dados disponíveis publicamente) com um processo automático para gerar entradas e rótulos desses textos. Mais precisamente, ele foi pré-treinado com dois objetivos:

Modelagem de linguagem mascarada (MLM): pegando uma frase, o modelo mascara aleatoriamente 15% das palavras na entrada e então executa a frase mascarada inteira através do modelo e tem que prever as palavras mascaradas. Isso é diferente das redes neurais recorrentes tradicionais (RNNs) que geralmente veem as palavras uma após a outra, ou de modelos autorregressivos como GPT que mascaram internamente os tokens futuros. Ele permite que o modelo aprenda uma representação bidirecional da frase.
Previsão de ordenação de frases (SOP): ALBERT usa uma perda de pré-treinamento baseada na previsão da ordenação de dois segmentos consecutivos de texto.
Dessa forma, o modelo aprende uma representação interna da língua inglesa que pode então ser usada para extrair características úteis para tarefas posteriores: se você tiver um conjunto de dados de frases rotuladas, por exemplo, poderá treinar um classificador padrão usando as características produzidas pelo modelo ALBERT como entradas.

ALBERT é particular porque compartilha suas camadas em seu Transformer. Portanto, todas as camadas têm os mesmos pesos. Usar camadas repetidas resulta em uma pequena pegada de memória, no entanto, o custo computacional permanece semelhante a uma arquitetura do tipo BERT com o mesmo número de camadas ocultas, pois tem que iterar pelo mesmo número de camadas (repetidas).

Esta é a segunda versão do modelo base. A versão 2 é diferente da versão 1 devido a diferentes taxas de abandono, dados de treinamento adicionais e treinamento mais longo. Ela tem melhores resultados em quase todas as tarefas posteriores.

Este modelo tem a seguinte configuração:

12 camadas repetidas
128 dimensão de incorporação
768 dimensão oculta
12 cabeças de atenção
Parâmetros 11M

Você pode usar o modelo bruto para modelagem de linguagem mascarada ou previsão da próxima frase, mas ele é mais destinado a ser ajustado em uma tarefa downstream. Veja o hub do modelo para procurar versões ajustadas em uma tarefa que lhe interesse.

Note que este modelo tem como objetivo principal ser ajustado em tarefas que usam a frase inteira (potencialmente mascarada) para tomar decisões, como classificação de sequência, classificação de token ou resposta a perguntas. Para tarefas como geração de texto, você deve olhar para um modelo como GPT2.



In [ ]:
# Instalando os modulos
!pip install torch transformers datasets matplotlib numpy scikit-learn pandas scipy

## Carregando os modulos

In [ ]:

from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    AutoTokenizer,
    AutoModel ,
    Trainer
)
from sklearn.metrics import (
    accuracy_score ,
    f1_score ,
    classification_report ,
    roc_curve ,
    auc
)
from scipy.special import softmax
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

## Carregando o dataset

In [ ]:
id_data = "dair-ai/emotion"
dataset = load_dataset(id_data)

## Entendendo os dados

In [ ]:
# Estrutura do dataset
print(dataset)

In [ ]:
# Total de classes do dataset
classes = dataset['train'].features['label'].names
classes

In [ ]:
# Alterando o formato do dataset para um tipo pandas
dataset.set_format(type='pandas')
df_pandas = dataset['train'][:]
df_pandas.head()

In [ ]:
# Criando uma tabela para cada classe correspondente
df_pandas['label_name'] = df_pandas['label'].apply(lambda x : classes[x])
df_pandas.head()

In [ ]:
# Verificando o balanceamento das classes
total_classes = df_pandas['label_name'].value_counts()
total_classes

In [ ]:
# Resetando o formato original dos dados
dataset.reset_format()

## Carregando o tokenizador do modelo

In [ ]:
id_model = 'albert/albert-base-v2'
tokenizador = AutoTokenizer.from_pretrained(id_model)

In [ ]:
# passaremos a quantidade de batchs dos dados
# Para aplicar essas a tokenizaço de todos os dados
# basta usar o metodo map()
# Função para tokenizar o dataset
def tokenizador_lote(batch):
    temp = tokenizador(
        batch['text'],  # Aqui, 'batch' deve ser um dicionário com uma chave 'text'
        padding=True,
        truncation=True,
    )
    return temp


## Tokenizando dados

In [ ]:
dataset_tokenizado = dataset.map(
    tokenizador_lote,
    batched = True ,
    batch_size=None
)

## Carregando modelo

In [ ]:
model= AutoModel.from_pretrained(id_model)

In [ ]:
model

## Configurações iniciais para o ajuste fino

In [ ]:
# Armazenando numero de classes
numero_classes = len(classes)
# Inicializando plataforma CUDA
device = torch.device(
    "cuda" if torch.cuda.is_available() else "cpu"
)
model = AutoModelForSequenceClassification.from_pretrained(
    id_model , num_labels = numero_classes
)

In [ ]:
device

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

## Configurações do treinamento

In [ ]:
# Tamanho do lote
batch_size = 15
model_name = 'albert-base-v2-emotions'

training_args = TrainingArguments(
    output_dir=model_name ,
    num_train_epochs=4 ,
    learning_rate=1e-5,
    per_device_train_batch_size= batch_size ,
    per_device_eval_batch_size=batch_size ,
    weight_decay=0.01,
    eval_strategy='epoch' ,
    disable_tqdm=False

)

## Computação de métricas

In [ ]:
def computer_metrics(pred) :
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels,preds,average='weighted')
    acc = accuracy_score(labels,preds)
    return {"acurracy" : acc , "f1" : f1}

## Treinamento

In [ ]:
trainer = Trainer(
    model = model ,
    args = training_args ,
    compute_metrics = computer_metrics ,
    train_dataset = dataset_tokenizado['train'] ,
    eval_dataset= dataset_tokenizado['validation'] ,
    tokenizer = tokenizador
)

In [ ]:
trainer.train()

## Avaliando modelo

In [ ]:
pred_ouptus = trainer.predict(
    dataset_tokenizado['test']
)
pred_ouptus.metrics

In [ ]:
y_pred = np.argmax(
    pred_ouptus.predictions,axis=1
)
y_test = dataset_tokenizado['test'][:]['label']

In [ ]:
y_pred[:5]

In [ ]:
y_test[:5]

In [ ]:
classes

In [ ]:
print(classification_report(y_test, y_pred, target_names=classes))

In [ ]:
report_dict = classification_report(y_test, y_pred, target_names=classes, output_dict=True)
report_dict

In [ ]:
report_df = pd.DataFrame(report_dict).transpose()
report_df = report_df.round(4)

In [ ]:
# Exibe a tabela
print(report_df)

In [ ]:

def plot_multiclass_roc(y_true, y_probs, class_names):
    n_classes = len(class_names)
    y_true_bin = label_binarize(y_true, classes=list(range(n_classes)))

    fpr, tpr, roc_auc = {}, {}, {}
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    plt.figure(figsize=(10, 8))
    for i in range(n_classes):
        plt.plot(fpr[i], tpr[i], lw=2, label=f'{class_names[i]} (AUC = {roc_auc[i]:.2f})')

    plt.plot([0, 1], [0, 1], 'k--', lw=1)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Falso Positivo')
    plt.ylabel('Verdadeiro Positivo')
    plt.title('Curvas ROC por Emoção')
    plt.legend(loc='lower right')
    plt.grid()
    plt.show()


In [ ]:
pred = trainer.predict(dataset_tokenizado['test'])

In [ ]:
# Aplica softmax sobre os logits
y_pred_proba = softmax(pred.predictions, axis=1)

In [ ]:
plot_multiclass_roc(y_test, y_pred_proba, classes)

In [ ]:

def get_roc_dataframe(y_true, y_probs, class_names):
    n_classes = len(classes)
    y_true_bin = label_binarize(y_true, classes=list(range(n_classes)))

    dataframes = []

    for i in range(n_classes):
        fpr, tpr, thresholds = roc_curve(y_true_bin[:, i], y_probs[:, i])
        roc_auc = auc(fpr, tpr)

        df = pd.DataFrame({
            'class': classes[i],
            'fpr': fpr,
            'tpr': tpr,
            'threshold': thresholds,
            'auc': roc_auc  # mesmo valor repetido pra cada linha, pra facilitar agrupamentos
        })

        dataframes.append(df)

    return pd.concat(dataframes, ignore_index=True)


In [ ]:
roc_df = get_roc_dataframe(y_test, y_pred_proba,classes)
roc_df

In [ ]:
roc_df.to_csv("curva_roc_por_classe.csv", index=False)